# Cactus reduction interface

Here you will find a step by step guide to create and test a Cactus thorn that uses the Cactus reduction interface to compute the approximate center of mass of a distribution of matter on the grid. 

This notebook is geared towards users who are already familiar with the introductiory tutorials, in particular https://github.com/einsteintoolkit/jupyter-et/blob/master/CactusTutorial.ipynb  and https://github.com/einsteintoolkit/jupyter-et/blob/master/CreatingANewThorn-WaveEqn.ipynb .

If you find something that does not work, please feel free to mail users@einsteintoolkit.org.

## Notebook setup
This notebook is intended to be used online on the Einstein Toolkit tutorial server, offline as a read-only document, as a jupyter notebook that you can download and also in your own docker container using `ndslabs/jupyter-et`. To make all of these work some setting need to be tweaked, which we do in the next cell.

In [ ]:
# this allows you to use "cd" in cells to change directories instead of requiring "%cd"
%automagic on
# override IPython's default %%bash to not buffer all output
from IPython.core.magic import register_cell_magic
@register_cell_magic
def bash(line, cell): get_ipython().system(cell)
# Some versions of OpenMPI prevent oversubscribing cpus, which may happen if simfactory's
# number of cores detection is imperfect.
# OpenMPI by default pins MPI ranks to specific cores, which causes issues on shared
# system like the tutorial cluster.
# OpenMPI contains a bug affecting MPI calls with large amounts of data on slow systems,
# which can lead to hangs (OpenMPI issue 6568).
import os
os.environ["OMPI_MCA_rmaps_base_oversubscribe"] = "true"
os.environ["OMPI_MCA_hwloc_base_binding_policy"] = "none"
os.environ["OMPI_MCA_btl_vader_single_copy_mechanism"] = "none"

<b>Note:</b> By default, the cells in this notebook are Python commands. However, cells that start with <code>%%bash</code> are executed in a bash shell. If you wish to run these commands outside the notebook and in a bash shell, cut and paste only the part after the initial <code>%%bash</code>. 

# The reduction interface

Cactus provides functionality to compute "reductions" of the values on the grid,
similar to the `MPI_Reduce` function defined by MPI. These operations include
finding the maximum or minimum value of a grid variable, or computing sums or
norms of grid variable data.

The exact set of operations is not defined by Cactus but instead determined by a
combination which driver thorn (`PUGH` or `Carpet`) and reduction thorn (`PUGHReduce`
and `LocalReduce` or `CarpetReduce`) are active. Among the more commonly used 
reductions provided by both drivers are:

<table>
    <tr><th>name (<code>PUGH</code> / <code>Carpet</code>)</th><th>description</th></tr>
    <tr><td><code>min</code> / <code>minimum</code></td><td>minimum value on the grid</td></tr>
    <tr><td><code>max</code> / <code>maximum</code></td><td>maximum value on the grid</td></tr>
    <tr><td><code>norm_inf</code></td><td>infinity norm the grid, i.e. $\max |\psi|$</td></tr>
    <tr><td><code>sum</code></td><td>volume weighted sum on the grid, i.e. Riemann sum</td></tr>
    <tr><td><code>norm1</code></td><td>volume weighted $L_1$ norm on the grid</td></tr>
    <tr><td><code>norm2</code></td><td>volume weighted $L_2$ norm on the grid</td></tr>
    <tr><td><code>average</code></td><td>volume weighted average on the grid</td></tr>
</table>

Note that there are no equivalents of MPI's `min_loc` and `max_loc` operations.

The `sum` like reductions use a volume weighted sum defined as
$\texttt{sum}(\rho_i; t) \equiv \left(\sum_i \rho_i(t) \, \Delta V_i\right) / \Delta V$ where 
$i$ runs over all the grid points, $\Delta V_i$ is the volume of cell $i$ that
contributes to the sum and $\Delta V$ is the volume of a reference cell on the
coarset refinement level. For mesh refined simulations $\Delta V_i$ depends on
the refinement level a cell is on and
whether a given cell is (partially) overlapped by a cell on a finer refinement
level.

With this definition of `sum` reductions Cactus ensures that

$$
\texttt{sum}(\rho_i; t) \, \Delta V \rightarrow \int \rho(t, x) \, d^3x
$$

as resolution is increased, where $V$ is the (constant) simulation domain.

## Using reductions

Reductions can be used either during output as options in the
`IOBasic::reductions` parameters, or at runtime via the `CCTK_ReduceGridArays`
(`PUGH`) or `CCTK_Reduce` (`Carpet`) calls.

When used for ouptut the output thorns will compute the requested reductions for
all output variables and write the result to disk / screen.

The programmatic functions `CCTK_ReduceGridArays` and `CCTK_Reduce` allow
application thorns to do the same for example to write custom output files or to
store reduction output in grid scalars for later use.

In this tutorial we will use only on the `CCTK_Reduce` interface offered by
`Carpet`.

```C
int CCTK_ReductionHandle(const char *reduction_name);

int CCTK_Reduce(const cGH *GH,          /* Cactus grid hierarchy */
                int proc,               /* MPI rank to receive result, -1 for all */
                int operation_handle,   /* obtained from CCTK_ReductionHandle */
                int num_out_vals,       /* number of output values per input field (must be 1) */
                int type_out_vals,      /* CCTK_VARIABLE_INT, or CCTK_VARIABLE_REAL, or CCTK_VARIABLE_COMPLEX */
                void *out_vals,         /* point to array of size num_out_vals for results */
                int num_in_fields,      /* speciﬁes the number of input ﬁelds */
                ...);                   /* variable indices of grid variables to act on */
```

## Example use

This fragments shows how to compute the `sum` reduction of `GRHydro::dens` which
is, up to the normalization factor $\Delta V$, the rest mass
$M_0(t) = \int \rho_*(t,x) \, \sqrt{\det g_{ij}(t,x)} \, d^3x$.


### Code
```C
const int sum_handle = CCTK_ReductionHandle("sum");
if(sum_handle < 0)
  CCTK_VERROR("Could not obtain 'sum' reduction handle: %d", sum_handle);

const int dens_index = CCTK_VarIndex("GRHydro::dens");
if(dens_index < 0) {
  CCTK_VERROR("Could not obtain variable index for GRHydro::dens: %d",
              dens_index);
}

CCTK_REAL rest_mass = 42.;
const int ierr = CCTK_Reduce(cctkGH, -1 /* proc */, sum_handle, 1 /* num_out_vals */,
                             CCTK_VARIABLE_REAL, &rest_mass, 1 /* num_in_fields */,
                             dens_index);
if(ierr < 0)
  CCTK_VERROR("Could not compute 'sum' reduction for GRHydro::dens: %d", ierr);

CCTK_VINFO("sum(GRHydro::rho) = %g\n", rest_mass);
```

### Schedule

```Python
schedule restmass_computerestmass IN CCTK_ANALYSIS
{
  LANG: C
  OPTIONS: global
} "compute rest mass"
```

## Computing the to be reduced-grid function values

```Python
schedule restmass_rho IN CCTK_POSTSTEP
{
  LANG: C
  OPTIONS: local
} "compute density"
```

This requires that:

* the grid function of interest must be computed in each timestep
* sufficiently many (three usually) timelevels are avaiable for interpolation in time
* the grid function must be checkpointed

otherwise the naive method of computing a grid function just before calling
`CCTK_Reduce` tends to either fail with an error message from Carpet or produces
subtly incorrect numerical values.

## Mesh refinement

Mesh refinement, in particular subcylcing in time, adds complications to
computing reductions (and any other "global" quantity):

* `LOCAL` scheduled functions are called once per grid patch, making it impossible to call MPI routines from them
* different refinement levels exist at different times only, requiring interpolation in time
* `sum` like reductions have to take relative size of grid cells into account

<!--
#FIG 3.2  Produced by xfig version 3.2.7b
Landscape
Center
Metric
A4
150.00
Single
-2
1200 2
6 270 1073 1095 1350
6 270 1073 450 1343
4 0 0 49 -1 32 18 0.0000 4 195 165 270 1343 S\001
-6
4 0 0 49 -1 0 15 0.0000 4 210 660 435 1305 (t=4.5)\001
-6
2 1 0 2 0 7 50 -1 -1 0.000 0 0 -1 0 0 2
	 0 2340 6750 2340
2 1 0 2 0 7 50 -1 -1 0.000 0 0 -1 0 0 2
	 0 4500 6750 4500
2 1 0 2 1 7 50 -1 -1 0.000 0 0 -1 0 0 2
	 1350 3420 5400 3420
2 1 0 2 1 7 50 -1 -1 0.000 0 0 -1 0 0 2
	 1350 2160 5400 2160
2 1 0 2 1 7 50 -1 -1 0.000 0 0 -1 0 0 2
	 1350 1260 5400 1260
2 1 0 2 0 7 50 -1 -1 0.000 0 0 -1 0 0 2
	 0 270 6750 270
3 2 0 2 0 7 50 -1 -1 0.000 0 1 0 3
	1 1 1.00 60.00 120.00
	 810 4500 1350 2880 900 1440
	 0.000 -0.500 0.000
3 0 0 2 0 7 50 -1 -1 0.000 0 1 0 3
	1 1 1.00 60.00 120.00
	 810 270 360 630 540 1080
	 0.000 1.000 0.000
3 2 0 2 1 7 50 -1 -1 0.000 0 1 0 3
	1 1 1.00 60.00 120.00
	 2610 1260 1800 630 990 1080
	 0.000 -0.500 0.000
3 0 0 2 0 7 50 -1 -1 0.000 0 1 0 3
	1 1 1.00 60.00 120.00
	 810 2340 270 1890 540 1440
	 0.000 1.000 0.000
4 0 1 50 -1 0 15 0.0000 4 225 810 4680 3330 rho_p_p\001
4 0 1 50 -1 0 15 0.0000 4 165 510 1440 3330 t=3.5\001
4 0 0 50 -1 0 15 0.0000 4 165 510 90 4410 t=3.0\001
4 0 0 50 -1 0 15 0.0000 4 165 510 90 2250 t=4.0\001
4 0 1 50 -1 0 15 0.0000 4 165 510 1440 2070 t=4.0\001
4 0 1 50 -1 0 15 0.0000 4 165 510 1440 1170 t=4.5\001
4 0 1 50 -1 0 15 0.0000 4 165 330 4680 1170 rho\001
4 0 1 50 -1 0 15 0.0000 4 225 570 4680 2070 rho_p\001
4 0 0 50 -1 0 15 0.0000 4 165 510 90 180 t=5.0\001
4 0 0 50 -1 0 15 0.0000 4 225 810 5760 4410 rho_p_p\001
4 0 0 50 -1 0 15 0.0000 4 225 570 5760 2250 rho_p\001
4 0 0 50 -1 0 15 0.0000 4 165 330 5760 180 rho\001
4 0 0 49 -1 12 15 0.0000 4 195 2100 3150 720 sum(rho;t=4.5)\001
-->

<img src='data:image/png;base64,
iVBORw0KGgoAAAANSUhEUgAAAsoAAAHbCAMAAADlKkzRAAABMlBMVEUAAAAHB/9YWP+ZmZmVlf+Pj49mZv9/f38cHP+IiP8+Pv+Pj/+qqv9nZ2dlZWUPD//MzP8WFv8xMf/u7v9RUVECAv9ubv9DQ0Pu7u41NTUxMTEQEP8rK/8vLy8yMv8hISEfHx8dHR3Ozs4bGxvMzMwXFxcDA/8eHv9UVP8PDw8NDQ0JCQl2dv8HBwcDAwOwsLCurq6qqqqioqIEBP+IiIgLC/9cXP+Zmf92dnZycnJubm67u/9mZmZeXl5WVv/d3f9WVlZUVFQMDP9QUFBdXf////9KSkpGRkZ/f/8+Pj46Ojo4ODgyMjIuLi4sLCzd3d0GBv8hIf8oKCgkJCQeHh4oKP9DQ/8YGBgSEhIQEBBKSv9lZf8KCgq7u7sICAgGBgYEBAQCAgIAAP82Nv9RUf+iov8Y3In6AAAgAElEQVR4nO2dcaO7ulnHo1Xn0cqqqKDVMhVnVTatVhvXqxWV6RSHnbMOO3a9qO//LUgCbSnQUwgJhPb7+eP2/Di05dLPSZ88eZKQAICXgIx9AQDIoYPKtrqrAKA3rVS2GGF0/XfiW5YRffIEAO6Jtpal+C3aqDwjDKP0700QbGNX3VWBV8M1yNAqbxrOcTZJxrUVjmJuteOovC7wYiRDqzxreL+Zcf9vSjb5w0zVRYHXY2iV3bjh/ayQJuV/x4Q30BviK7wu8GIMrPImJnEtOk9YoOxsr/92CbkcV3xt4AWwZzSgFuUqU8uieR5s41PLSJ48tSN3OkZbEibVN4g2NCz3+jakCJIJQRIDPGHmEItaJGYq+zQziXtkhHZgG5Ij1PuWlX8L8Ga4gObHI4tc35devikIkfxnBV6QhMRusNnkj0GUSc0UYtkv25ErUIPKEb1xfS+DhMVPUBl04BIjF488Ko1jfmgrN3xuULkRO76cubmpjMwyeEaDyrdDUntbbVUOfHI9J/8pQrcPPKdR5eIbXrnKjQFGQOPrUy4ZjDAA4AmNKhfWKVe5oduXYV2zyH4eI29LvwTgAQ0q20UvK5LbGDaobCc3ruk2m7+76/KTeJLZRy4OPKep22fkKbmZ3GxcZbSPOLZ7X4bhxk4mse3P+K+5vlZss5/RKIPnXOLQzUXlzCA7ZBkDOzQ+fWZXKtGKVa6A49hOdoj6/BuhUNmm4WYWw2TwnFmYRamb4jFyfUKYSrYRUxpKNqgaeCf1qKEUZ7jFD1GSoBAf9MCujSr3Bgk18CJAZfAiQGXwIkBl8CJAZfAiQGXwIkBl8CJkKrsJABPH5ipbBICJk3CVZ1Q2f/0t8g3pLwrAYyI1sfIsrhZyAKAcBSpTgqI5MDzyVTZIjGWLwPDIVtm2CNZFBGMgWWU7JCFMBmMgV2U3MxmFzGAUpKrsxsSCyWAcZKq8QRIOjIdElWdIwoERkacyJVg7HIyINJUNEjdt/gDAQMhS2UA6GYyLHJVtHyaDkZGiMgZGwPjIUNnGwAgYHwkqw2SgA/1VhslAC3qrDJOBHvRVGSYDTeipcuTAZKAH/VS2QxQQAU3opXJmsuqNiwFoSS+VEScDfeijsgGTgT70UNkgMfaIAtogrvIWFURAJ4RVnmGLa6AVoiq7MeaMAK0QVNkOMY8P6IWgyhQJZaAZYionSF4A3RBT2cl3bAcyaOg9uzL3Gn20cWmUrxdfvH00+SECIZURXkiExvVjMWn/uTx10H/wYjRfL754ezr5j1REZTsmCC9kETVmgpL2n4v1pP+dxLMHKt/J23wdU0JEZYpyOHkYTtNRiSo79MGLVdrh5guZECIqj98oH+aetz5lnL3d3hz5Ymo8DnQjFpi65eA4qiY183CB2Rcl5dvMN0Fq4onK1Hn0d1FR2Z16syyg8mzcRtncnY/pHStvPuYF3RP5LAA1LOarw6PUyOGL6RnEMggJ2e9Ly51uL83CLHtWYNOYOOyZCbHZxl1WYb1tZNEz8a9qU8soXsIpdkh6VEQQxcljle1kW/qrC0PB/2VNEFDZJ+OtqGV+nHJ5P/b7fRDs93NvzcRenhejXdM9DlsRxPXZ7ltBsuW3N0nCTGXXyDy2nZj959aQhpev9Sg72Q3Djc2/6BMS0ij7s4i5sXZ2nL3oxdeEXNfny17buGxcx17utv1X3p0z/IfRSvZnE1txfJWZkmknMQRUjkfbgXW3zqw9nueVkOLAj69241zUPQnJ1cgNuUjEgwDKvKSOfffVTvzSU2Pj9jP/0c63dbGK9vISzkakVABzF2CUtxPlr8Py/w9U3hguv6pLWz8jEjOAI9Bdy6R094fk4GXN79Jrbn3Nj+yXRx1kjn0uWW7avcrW3X84bilUTkrJ+rsnupcgxL5sWBTRWzzxaazMvg6e9CGty99PMvFahO4qb0f5Pz6cs4b39FlMvD9pIXMWW2Tf2zSX76nKyZ3KQe1n/sTZNbfQpO3dsW1pV0b2z5g1z9tPaxivr/5+KvsjfA8xkZfe4clZTObVfpAL+hQ32YZ5WPtUZbuNyttrf8zvqPJ1z+dPkhPXt00mPoTbXWVr8FDZ9JjIbXJu+1WarkdNzl0ybT6Pwp6qHMS3Hx+qfA0wAidXMspEvXbRnuWVnwUY14vZvl2sTBoGWlViesuWIjN22ckfSq/ncy5iUd6SJnlSwI0fquzf7uZDlYOw6J1s84RGJl2pcqCTyq5z3R90lv/R2c6lLTZG687LQUDlYQfrMzfT87PQooSZxSKn8RpmK94y3WZxLhjLSdg0jo0oiAyWpqPX/+Rc0wZZVEJYYMsb4Cj7OWKFQCHPMrgxSynb9JIFzcLx2+jJlscy7sMaCjspNbfWteYioIS17FF4nWocT3wMV3OVecTQQWT+nGO6HG3MxDAcEoakMDnYxE5ILNciIfMv635RZiEl8TU+KASy4yKq5XeXpYd93vrmZ2YvkL2OczGS3A1S+ex3xH8UHRjlWJmW2vONQxyH+JcrmXouTm+VWQsr0I9jsfWIEbOblEau7WsV5QNouyqAqPQ6m9JfQvG7R6PaNe7OdJPS2Lg19dI4nVX+yOJesezafJkedRn+e4bT+XudqsiaJZOfdayvyiy2aN3bq2KesidLvRxlJJ1HnCxHwQjzdtpJ5UDfDAYLEnrliD/GTsupIxq7MFFPuqvsDJGzmffPqS2O6WoqQQaQgMgQifJGwVx3z1s0vMppxEwGGJzuKhvKkzas0ybFQW8yATPoT3eVqeKhetYkC3f3KuyWYw6XgEERKfJUOirEomR5JUGLVbpEwPwedFfZJgonzvAoWWY7mr2gYG4aTAyBdISjbuKMtCi5RBYwj1lfBIZCQGVD1dw+mVFyiV2anqW/KNAOoRnXagaG9lmTrKRwfrF82dEScENAZVfJ0LXKGqCs87eCy6+O2JIu0i+DpxrUjWeYSGS8PiJWKpjdlzXJp97De59gnjHy9+qIqCx9zvXhpD7LkP2xICn30oioLDtY3i2HKPzZYRT7tREKe2OZmWWegpP3co+ZL5GUe2WEVJaZWWYz8QZau2KxRCLjhRFSWWJmedBpeAck5V4YIZUjWWUY5gD9vbv3Q1LudRHdVidyJcQY+8Fnk7KkHFx+TcRU/oL8mPxm7/ceZY4/EsyvioDK0dd++BUh5Bs93/lwGmdJLCSYXxQBlZNiHR3h9+QJi/kgyeQmdnD5JRER8otv91J5kf4TbxvPYyUTMFjykgjt2/cjrrLgxGvzp9L0/1ajzu3AYMkrItS28hDjjwRrin4l39Vp1DzCAi6/HoIZDBZiiGXjfu6bzOSvCz1XHgtMxX45xFTmIYbQiN8vcpPTb/6v0Pt+RjczUY3/cgh23liI8dcCz1v8W75r5K/9cp/q5PPp7p9sW9XTquMLmm/t8uHjdHp+1rQQzUNkIcb3uj/L/Pm8Tf6DXn2+eXr3Mez4a3aOfd/a5cU5hcoFWYjxg+7P+h9u3S+0roRrHJc7nO4/huN8n9G9lWcuv+0g9h4qX/kT8oedn/MPf57++b/9dHvpdo23+7S4+xh2wrkIttzLu7oMlUt80TmF8Tdp+vVf7/Cdvlg23e6Pj+DuYzitPOF65/ctLoLKJWbf77Z2nPlbf/Eff9nlCWz1uHrnZJEdKX8MexayHEWLOd7OZXPnBd7J4yp7p1O+gs587Z3OGuzd2Q9xlW232+YVh92+mzWHj3S1r95gk03MLqt8mHsrkV5fwZu5vDumJ++ULpnKay+7c+y+nbPur3mefGHKgNsOHrrmC/iX4D694QWBx2549cuRzdgW/SDerOhzz/505/P8MThkUgdeyv6YzWM68XZZ5x00ucoH78Y+2K/ZL+px3jldib7Lx1sVyl1i5OIxTYNgueRHPqYePWuv8h3mkbfs9ZtuLlPht3mros+6yrcj4rdQC6al8nnNW+f06HmVlN66x+fwTi43qVx8oUFlddQDjNMtbq4Edt6yxxu9kctNKhcKQ2V1NHX7OA1R3Wnd553ex+W6ymbRLBzEuxt6MJTKIssZ8dtt7m9cgori41jc0mhmz+7327jc0O0755nM3dTvgGCRJ/1MTeNuSTmLEUZPntPEIj2ai6ZMWf4xLNL0ECyWbHlxc933Y3iXSVKXyHh+UfnAalEWrCJl6pMRnqlsx6RE4ajrf2bl5m51xBl/IhsYdK2uLp8eDH2UVDaP2Tneun9KdP8WE0t2bOPwefF4WLBNjPaBeV563mryf8lPW+UtITFX0E5o4Wjk5NP6mmswIutOZWeTZPAnuGFXlz+vd1vw3wpVxTW92lu43IxZG1WdIM8DDItcN+rb5o6GM/4wa16a1rpbs3ZWLtSgWu9u/84uvwLPVXaz+KCYkWpzR2mcH48bVd5ug7LKVkhv01kj5XsK9wIuT5oW3T5Krqsd8i2BY97QbmISW3WZ3exQSWU+N9u57iQcqtiQRx5wecq0UNl27qakzvI9rqMtCZNaI2tb0Z3K0YaG5BahGKKLZwwEXJ4wbZJxm6znd1PQKKKEhBmblNIbTHfKwujK/g5ZP5Dk0bXyrd57A5enS6u8sk+If/2HVVY5ojeywwk/rbZViXGJUDaKdq+UB1yeLK1UjuLSslp3Kt9jOzzbVvuFfdnpr+E5ugGXp0q70T5Kbjm1T1Q2fN48E4fS+5jYb6/yfmx+5u/S/xr7GqaAym0WhWinslEa3PgkwLBugfN9h7DI32XP8YMnpGAaaDc62ErlWVyaxvdpty9/zVrTaxUGt9i80hufnxv7AiaBduODbVSO4nLeoUjG5SrbyY1rTFGofJvGal8aaar3GAmYMm1UDu+iAjsfIglc4tjNe+vkKrtZV9GNncxd2y9ycYHl9LhUAD6jzWifcwuUN5T9O/+5VJxRec2rymx0xaD+pSm2tc/FgenyXOUNuQXKtuOy/xRtcfL50J3Lfl0OPAIqabs/AOo8VTmLJy7BMKUxz0S4gkbyPwQA1PBU5VKC7VpMsem2xFaBbcFkoI5nKtM7ky8rHroCLttU71IiMHEEp6kKzDoVmagKQGt0XjwAgA5AZfAiQGXwIkBl8CJAZfAiQGXwIkBl8CJAZfAiQGXwIoirHD0pjCtwfyj8DgB0QFzlv/+dVsXH7m/Pnp8EQG+EVf7db/1Lq6qKDflC9C0A6IDwdu0/JN9pd+bvkb8SfA8AOiCq8r+SL/+03ZnfI1+iThmoR1DlX/02afvMXyLkhyjwBMoRU/k7XxHyg5bnbv6b6L+8Fpg+Qir/bNYmt+7NRdm5X31N5G0A6ICIynxJom8370TS9Bb1hbcAkI6Ayu6XXM7W8e+/s7O/hXAZqKW7yvaPuMk/av2E7/LzsS4RUEt3lZ187vV3Wz8hn7T9bYyUAKV0Vzn5wVdMzfbD0clP2Plf/SdCDKASkW7f32Zm/nf7YQ+WwiC/j34fUIuAynZM/uwHP+7whD/+8h8RKgPVCKjMd3PostLQxg4s0jp3B4AQAhmMWGDzvdnznRsA6EV3lbcPVlX+HBH/AeiASDJOpNCNYpVwoJbOKidEaHXliMQiTwOgLZ1VNjpklMv4gs8DoB1dVc5aV7GhDsHWHICWdFVZPOZ1UB0HVNJVZfFMxEwo8wFASzqq3CM/LJSPBqAtHVX2e4zaIR8HVNJNZbtPSi1CIQZQSDeV+8W7FvJxQB3dVO4TX7CFijDzGiijk8p9QwQHHT+gjE4qb3t23NDxA+ropLLfc5QDhRhAHZ1UjvsuLI5CDKCMLnK6vbtt6PgBZXRRedY/1EXHD6iii8q0/wQ9dPyAKrqobPVvUtHxA6roorIjYTspdPyAIrrYKaOGAh0/oIhOKsuwEB0/oIbBVRZbewCAZwyusi06ORCATxlcZeEp2wB8SieVpSTSMPUaKKFbXllKaBBi6jVQQBeVDTnLcWLqNVDBwDUYDDuW07oDUKaLypGkKNcgWymvA0CJTmPRkoY3XEy9BvLppLKsujZ0/IB8Bp2megEdPyCfbsVuklay6Dvidz6V/7Vfn07nQ78rejcOH6fT87OmRTeVE+JIST70G/Gbp+WPYZfOg+Bjueh7TW/F4py+ucpZsywlWm474jdvOng4lT+Gw/LMHo5HCZf1RuzfXmXhlcIrtMuF7Bpv92lR/hi8dJ4/7GRc19sAlQMqp6aoVS5ksWy63R8fQfljWKY8TJ6naxnX9TZA5cB2pIxvtJnjN1+my3rnZJEdKX0MizTlj/viEXyOufMC7+Rxlb3TyTPZwfnaO533tVM/vNspE0Bgh6hYxjhJizl+h490ta/eYPN0uFN5nhZBcpoiifGc3TE9ead0yVRee94qZR2N88oMzHM1QstOXa3W3jJdTcTl7jNPqZTh602L5fP5l2DW3F7xspiY3fCSyt7l5zStNSugzj5dLoL5PH8MDpnU2S1k2R/zWL2B+Snmkt32KSAwiTqUMsDRouPHVT54N/bBnkfEUFmYS4xcPLKwbLnkRz6q0XNxyi6dSHJIQOUoljFQ0qLjV++amEf+ZXcXYFxVRma5BXWVb0fSxlPNqfRCRJa2SIiEwuUWg+B1lc9r3jqnR887XM7Jb/RhKjd8ZJpUXuW/eqBy7biuCK3SMiOxyD7X91hP/x7qAcbpFjdfwolrBmPV+4regSaVC1XfUuUsOuifxni+b1pTt49Tbq3XudQfU+mcjExdZbNoFg7VxuAaYEwkYy+4dpZBwt6jfk/38eP30tzfuKTbio9jseAnfbCf18jFtaKh23fmGbmse7drPHU3lf606DJwVn+X6bPBlkV6NBdNZRj5PV7kqeTT0mQ/o1FuxSUQm19UPgTmivWYzdW5dipPxh2rx3VFVGU77O3y845fFho33sc7lU1vNd8tYXIrdqssTJsXj4fFOk3X+8A8Lz1vVbuDmfWnc8NxXRFenFOCy88nk+w/jRoWxW8P+/1EBqR0xayNqgZ5gLGYSHDBEF9nNnNZeL/rHEwm0ZqpVRy1VdkoF8QlvmUZke30VBHLx+lGKVt0kqtytLVUr0bcUuW7ZZFnLCO8jV037ukylo/TGakqu4byhbWrKjcPW0RW6UKimAvsOEFfl7FuuMaw8ShPXjFAMrTKs+b3s8r7nBW769CsSe3rMtYNfxuGVtmNG99vuy0vSFsMbfAyzZ4uP00tg1dhYJU3MYkbonPXKq+t7BJSXBt77OeyiwVqXx17RgNqUa4ytSyad/Q3PrWMairW3tLSKZ25UznakjCp5XptKyqrvLmMbBDeOvdzOSTVuqQTmAbtpgXPHGJRi8RMZZ/SotbdCO3Arnb6s1PD0Kex6HjFfYDBvwWy5vYKqymm7B1vKl8nqpJ8hMPNLlY4p7atVS2nYBq0HAVMWBXlZpM/FnM6KW/AbKcyRJafYseCC1Q0qBzRGwlLIvPzHqrca9xP1tJdQFcuMXLxyKPSOJ+jvK2Ez8UpM0EnGlS+x87XIyoHGFeVi+Cgj8sWVkJ8bRpUvh1qtM8mYkPQT1U2fN48E4fS6O4Cots79nAZg9cvTqPKRWe/WeXq4bY8DTCsW+Cc3F1AebEscZdtbHr92jSqXFinXOVqty8/79Zc+7nTdz22zGVHbI4UBq9fmwaV7aJVrO6icA0wxMrUGlS2kxvXwbj8XVyXn8THNfy7kTrbEJzv12ZBDDBdmrp9Rh5Vziqt2LXbJ9Z9qoz2Ecd2G8sw+Lu4eSrZYgVtbjVlIuoyNtl5aS5x6OaicsS/w1nWLTSqp/JknCPYe6qEJVlk3PxCZZVtGm5mcS35l7ks8udEEWG8MLMwi1I3xWPk+oT4CfsOjykNqwZl1ltG/XBbqhF28ml9j1v8NkqShqaUEhErXUQYb4hdH1XmAYYrnpsVn0XSwIyIlAc5iDAAo2fFkVSVmcvdAx1EGG9MKVtmaaWyUHERIgzA0Utl5nLnwRJEGCAjsbKeYY8F3GSrHETdB/5QgA8kIF1lgUFsFOADCchXmbnccbAEU/xAfxSozAexOy3AjAgD9IcE5do3AKZJwlWmlny+/xudTv8J+WcFFwHeB1dNgNEdjJKA3uihcoJREtAXPVRGpSfojSYqGxL2nALvjSYqbzBbFfREE5UxWxX0RROVL3NfARBFF5XrK24B0AldVMaKW6AnuqgchCgpAr3QRmWUFIF+aKMyBvxAP7RROUA6DvRCH5Wt2gL4AHRAH5WRjgO90Edlt9/UcfDu6KNyEGt0LWB6aKQPxq5BHzRSGcEy6INGKiNYBn3QSGUEy6APOtmDYBn0QCeVUYYBeqCTyijDAD3QSeUANctAHK1UDrGEABBGK5UN9PuAMFqpjEESII5WKqPfB8TRSmX0+4A4eqkc6nU5YEro5Q7G+4AweqmMdZaBMHqpvEEKA4iil8o999ME74xeKgfYwQ+IopnKKFkGomimjoWV44AgmqmMKgwgimYqU+xJAgTRTGUUFAFRNFM5gcpAEO1URm0cEEMzlSOMkQBBNFM5gMpAEO1URsUyEEM7lXW7IDAVdDPH0u2CwFTQzRyoDATRzRyoDATRzRyoDATRzRyoDATRzRyoDATRzRyoDATRzRyoDATRzRyoDATRzRyM9gFBdDMHm7YDQbRTGZVxQAyoDF4EzVRG6T0QRTOVE2KMfQlgominMqapAjE0U3kGlYEgmqmMDVWBKNqpjIW2gBiaqWxBZSCIdipjP1UghmYqY31lIIpm6mDVeyCKXipjyTggjF4qY4coIIxeKmPfPiCMXipbxB37EsBU0UtlR6/LAVNCK3dsJDCAMFqpjBJPII5WKqOYCIijlco+KjCAMFqpjGFrII5O8mBiH+iBTipjrA/0QCeVMdYHeqCTyhaJxr4EMF10UhmLbIEeaKQyKjxBHzRSuecAiSnrOsA00UjldmVxu1V6PN9re2KsDmqu6jU5fJxOY1+DbDRSuVWofD553jFdlQ/tUsZZ0VW9JotzCpWV0RQqz6sHdl72H/OY7kvHjvN9BhrlTuyhsjoaQuVd7XZ7+X/TkuM7tMcCQGWF1EPlxfLB7fbSUht8Wnn75tPAY6CyOuxaqDxfpsvmzslxd/t5zwLl44fCK3spzCxE804eV9k7nTzegZ6vvdO51h6YH97tlBbHx0YblTe1svvDR7raNzS45nldPmvurdDra8vumJ68U7pkKq+97M6x+3ZemdlNTXe1U1ertbdMV2ar46OjjcoNBRj8S5C3ugW8z/dxTNPT/V08nNLqBwGa2afLRTCf54/BIZM6C9eyn6p96cupgblMvXbHx0YblcN6AQZX+eDdYPf6sJ+v643w+T4/Bx5xiZGLxzQNguWSH/moRs/FKbv02O742OiickSc2rGHXZMP9gnckbUR8q/pFamrfDuSNp5qtj0+NrqovG2oVX7cy17x78Qya83uq640qVx8oT1QtvXxsdFF5aZpfY0BBucuG5cfWaq+wtegSeVCSagshXoqLnjQ7ePUxT2tq0dAE3WVs0CBNxGHanfjGkis2x0fG01UrqfiguKemfsb16Z4xaVe3KIMs9r9Bs00dPvOeSd6V00CXbt3tcxG8/Gx0URlo2ku1CI9mov7Moz1mun7cc5/nYUZi+Uxu6PmGrm4dlwi4/lF5UNg8p6HuapmhYqk27Ht8bHRROW4ceOGUy3rtkrTtXfOveUqm8fsHG+tWQOhLbvsBnrz4vGwWGe3cx+Y56XnrWpZ4sz607nD8bHRQ+XNgwkk9Xq3RSnMWPCfUBXXH7NpVJUFEosOx8dGD5Ub4wswLo9yobpWIumhcnN8AYamlC06lZV9dFwrtFD5UXwBxgStsgCILzRkn3W6veqg6ifHR0cLlR3EF6A3OqjsYtlD0B8dVMYK4UACOqjslEuVPTANtMssa6Cye7eZTgqmAUb76tzHF3swDbQbYtVAZQdr0QIJjK+yi836gAzGVxn5CyCF8VVGfAGkMLrKiC+AHEZXGfEFkMPoKqO+E8hhbJVR3wkkMbbKqO8EkhhbZcQXQBIjqzxDfAEkMbLKPtmMewHgZRhX5cb1tQAQYVyVZ03rawEgwrgqWw3rdwIgxKgqN60PDoAYo6rctD44AGKMqrLTaldrANowpsooigMSGVNlFMUBiYypMgatgURGVBlFcUAmI6rsoygOSGQ8lTFoDaQynspbDFoDmYyncoikMpDJaCq7GLQGUhlNZSSVgVxGUznGSi5AKmOpjJlQQDJjqYyZUEAyI6mMSmUgm5FUpqhUBpIZSWUs3wlkM47KG2xvBmQzjsqoJALSGUVlVBIB+YyiMqanAvmMojKmpwL5jKFygumpQD5jqIw1lYECRlDZxvRUoIARVMaah0AFI6iMNQ+BCoZXGZVEQAnDq4xKIqCE4VVGJRFQwuAqo5IIqGFwlZFUBmoYWmWbxEgqAxUMrTLWJAKKGFplVBIBRQysMiqJgCoGVhmdPqCKYVVGpw8oY1iVKTp9QBXDqoyRPqCMQVXGSB9Qx6AqY80AoI4hVY6wZgBQx5Aqo7wTKGRIldHpAwoZUGV0+oBKBlQZnT6gkuFURqcPKGU4ldHpA0oZTmV0+oBSBlMZnT6glsFUttDpA0oZSmUs5AIUM5TK6PQBxQykso19gIFiBlIZq3cC1QykcojVO4FihlHZRacPqGYYlQ2yHeR9wBsziMrYsgGoZxCV0ekD6hlEZXT6gHqGUBmra4EBGEJlrK4FBmAAlbG6FhiCAVTGkspgCAZQGUsqgyFQrzJq7sEgqFcZE63BILRReRYSxyj33BLfsoyWRZuYaA0yoq2l+su5hcqGRalTTg3PyCbrzMXtImDU3IMM11AeZ1ZV3tTOmDEVbec2YBfFPCPhtCt2Q809YCRDqzyrv1/eqFJylbz4kbaKgWfE73F14GUYWmU3fvR+9Na4Fu3sppWkFsovAGNglTcxiR9E5861CXZJ/pyEtIizUXP/7thZgEotylWmlkXz9MHGp5ZRbeTsLS2d8vx4hU6IEVoAAAI8SURBVDsdoy0Jk6ZW1DZuLfDmYidpEQWj5v7NmTnEohaJmco+pWE+8muEduZUJULNTg1Dn8YktFsdr3LfsvJvgay5vcIDZXvrEGJdXodevinI89gBNfdvT0JiN9hs8sciNUv5+G85lXA7lTlDG16i4XiFBpUjeoO/V5RsfHItpOiiMmru355LjFw88qg0zocatpXwuThlVg1KHx2v0KByI9trZLy5qfw0s4ya+7enQeXboUb77Gon7NHxCm1Vzqx0L+fkz4med/sSdPrenkaVi/G2ZpWrhx8er9AmwODcsnHXDMbTuSGouQeNKhfWKVe51u3j0GsphZ9HDdunI9IRau5Bg8p20cuKKo3hNZDwG1+idrxCg8p2cuOWbguZuK7LT+IJNv9pLg7lF6Cx22fk2YBZ5Uv72r2rdLAeHa9QGe0jju1WyjB8n/m7Nfivub4Wa2zd556i/AJc49DNReXMCZt3vOzQqJ7Kk25ONev16HiFSvhhkVr6LCTEpwb/+ylUtmm4mcVPTUYmDrACYUI3xWPk+plMCRtyiykNqwZl1ltG/fDD4xWqkXRSb0jdW5zhFj9ESfI8CkYmDjzEbhhVZoGE2+DMo+MV1M0iQSYOdONRKrhlJZI6lZGJA88pZcuskrLlw6OrjNUvQEd0bZUpOn2gE4mV9QwbaiEeHa+iTGXsOAmGRZXKWP0CDIwqlbHjJBgYwvPG0vkz8p8KXhWAB9hcZYsAMHESrvKMquA7Sl4VgEaiAbdrB0At/w/991JAuhCnBAAAAABJRU5ErkJggg==
'>

## Computing the reductee on the fly

### Mixing compute and reduction calls
Not wanting to store each reductee indefinitely on the grid, we want:

```C
CCTK_LOOP3_ALL(compute_dens, cctkGH, i,j,k) {
    int idx = CCTK_GFINDEX3D(cctkGH, i,j,k);
    dens[idx] = ...;
} CCTK_ENDLOOP3_ALL(compute_dens);

CCTK_Reduce(cctkGH, -1 , sum_handle, 1, CCTK_VARIABLE_REAL, &rest_mass, 1,
            dens_index);
CCTK_VINFO("sum(GRHydro::rho) = %g\n", rest_mass);
```

which is impossible to schedule since either (in `GLOBAL` mode) we have no access to the data on the grid or (in `LOCAL` mode) we cannot call `CCTK_Reduce` to compute a "global" quantity.

### Mode changing macros

Carpet provides macros in its `carpet.hh` public header that allow user code to switch modes and iterate over all refinement levels and grid components:

* `BEGIN_REFLEVEL_LOOP(cctkGH)` ... `END_REFLEVEL_LOOP` to loop over refinement levels
* `BEGIN_LOCAL_MAP_LOOP(cctkGH, grouptype)` ... `END_LOCAL_MAP_LOOP` to loop of each coordinate map on the current level
* `BEGIN_LOCAL_COMPONENT_LOOP(cctkGH, grouptype)` ... `END_LOCAL_COMPONENT_LOOP` to loop over grid components
* and `DECLARE_CCTK_ARGUMENTS` to set the grid function pointers

`grouptype` is `CCTK_GF` for regular grid functions.

Carpet uses these same macros when traversing the schedule bins.

Using mode changing macros we schedule in `GLOBAL` mode and compute all data on all refinement levels, coordinate maps, and grid components as needed:

### Code
```C
#include "carpet.hh"

BEGIN_REFLEVEL_LOOP(cctkGH) {
  BEGIN_LOCAL_MAP_LOOP(cctkGH, CCTK_GF) {
    BEGIN_LOCAL_COMPONENT_LOOP(cctkGH, CCTK_GF) {
      DECLARE_CCTK_ARGUMENTS;
      CCTK_LOOP3_ALL(compute_dens, cctkGH, i,j,k) {
        int idx = CCTK_GFINDEX3D(cctkGH, i,j,k);
        temp_dens[idx] = ...;
        temp_dens_p[idx] = ...;
        temp_dens_p_p[idx] = ...;
      } CCTK_ENDLOOP3_ALL(compute_dens);
    } END_LOCAL_COMPONENT_LOOP;
  } END_LOCAL_MAP_LOOP;
} END_REFLEVEL_LOOP;

CCTK_Reduce(cctkGH, -1 , sum_handle, 1, CCTK_VARIABLE_REAL, &rest_mass, 1,
            temp_dens_index);
CCTK_VINFO("sum(GRHydro::rho) = %g\n", rest_mass);
```

### Schedule

```Python
schedule restmass_computerestmass IN CCTK_ANALYSIS
{
  LANG: C
  OPTIONS: global
  STORAGE: temp_dens
} "compute rest mass"
```

# Putting things together

To demonstrate all concept we will write a new thorn to compute the center of mass (normalized mass dipole):

$$
M_i(t) = \int \rho(x^i, t) \, x^i \, d^3x, \\
M(t) = \int \rho(x^i, t) \, d^3x, \text{and} \\
C_i = M_i / M
$$

using the rest mass density `HydroBase::rho` for $\rho$. Note: the "mass" $M$ defined this way does no correspond any physical quantity since it is defined without using the metric determinant $\sqrt{\det g_{ij}}$.

## Skeleton thorn

Cactus comes with the phony make target `newthorn` to create a compilable but
empty thorn setting up a skeleton of files.

In a shell you would use

```
make newthorn
```

then respond to the prompts as directed by the script. In this notebook, we
automate responses to the script's prompts.

We then overwrite

* `configuration.ccl`, `interface.ccl`, `schedule.ccl`
* `src/compute.cc`, `src/make.code.defn`

with our own files.

In [ ]:
# this assumes that you have a setup similar to CactusTutorial.ipynb
cd ~/Cactus

In [ ]:
# prompts by the newthorn script and responses to provide
newthorn_chat = {
    # prompt: response
    "Thorn name": "CenterOfMass",
    "Pick one, or create a new one.": "EinsteinAnalysis",
    "Thorn Author Name": "Roland Haas",
    "Email Address": "rhaas@illinois.edu",
    "Add another author?": "n",
    "Licence": "GPL",
}

In [ ]:
# this drives the chat, logging everything to the notebook
import subprocess

# remove any existing thorn by that name, the chat hangs otherwise
print(subprocess.check_output("rm -rvf arrangements/EinsteinAnalysis/CenterOfMass/", 
                              shell=True, universal_newlines=True))

# "PERLIO=unix" ensures that prompts are visible and not hidden by buffering
newthorn = subprocess.Popen("make newthorn PERLIO=unix",
                            stdout=subprocess.PIPE, stdin=subprocess.PIPE,
                            shell=True, universal_newlines=True)

prompt = ""
while not "Creating thorn" in prompt:
    prompt = newthorn.stdout.readline()
    print(prompt, end='')
    for tag in newthorn_chat:
        if tag in prompt:
            print(newthorn_chat[tag])
            newthorn.stdin.write(newthorn_chat[tag] + "\n")
            newthorn.stdin.flush()
(stdoutdata, stderrdata) = newthorn.communicate()
if(stdoutdata): print(stdoutdata, end='')
if(stderrdata): print(stderrdata, end='')

## interface.ccl

The interface file defines grid functions and grid scalars provided by
our thorn, it also states the name of our thonr's implementation.

Since we need access to the coordinates we inherit from `Grid`, which is usually
implemented by thorn `CartGrid3D`, and `HydroBase` to have access to `rho`.

We also declare that we use Carpet's provided header files `carpet.hh` to gain
access to Carpet's internal macros and data structures.

In [ ]:
%%writefile arrangements/EinsteinAnalysis/CenterOfMass/interface.ccl
# Interface definition for thorn CenterOfMass
implements: CenterOfMass

# Grid provides coordinates x,y,z, HydroBase contains rho
inherits: Grid, HydroBase

# use include file provided by Carpet describing its internal functions
USES INCLUDE HEADER: carpet.hh

CCTK_REAL tmp_rho_xi TYPE=GF TIMELEVELS=3 tags='checkpoint="no"' "integrand for center of mass"

CCTK_REAL center_of_mass TYPE=SCALAR TIMELEVELS=1
{
    M, Cx, Cy, Cz
} "center of mass"

## schedule.ccl

The schedule file schedules both functions and storage for our thorn. 

In [ ]:
%%writefile arrangements/EinsteinAnalysis/CenterOfMass/schedule.ccl
# Schedule definitions for thorn CenterOfMass

STORAGE: center_of_mass
STORAGE: tmp_rho_xi[3]

# ensure that HydroBase::rho has the 3 timelevels I assume it has
STORAGE: HydroBase::rho[3]

schedule CenterOfMass_Compute IN CCTK_ANALYSIS
{
    LANG: C
    OPTIONS: GLOBAL
} "Compute center of mass over grid"

## configuration.ccl

Our thorn uses Carpet's interal macros so must be linked against `Carpet` as well as have `Carpet` active at runtime. 

In [ ]:
%%writefile arrangements/EinsteinAnalysis/CenterOfMass/configuration.ccl
# Configuration definitions for thorn CenterOfMass

REQUIRES Carpet

## src/compute.cc

The scheduled function has to compute the three dipoles
$M_i = \int \, \rho \, x^i \, d^3x$ as well as a normalization factor ("mass")
$M = \int \rho \, d^3x$ and then set the grid scalar to the result
$C_i = M_i / M$.

In [ ]:
%%writefile arrangements/EinsteinAnalysis/CenterOfMass/src/compute.cc
#include "cctk.h"
#include "cctk_Arguments.h"
#include "cctk_Parameters.h"

#include "carpet.hh"

extern "C"
void CenterOfMass_Compute(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_PARAMETERS;
    
  const int sum_handle = CCTK_ReductionHandle("sum");
  if(sum_handle < 0)
    CCTK_VERROR("Could not obtain 'sum' reduction handle: %d", sum_handle);

  const int tmp_rho_xi_vi = CCTK_VarIndex("CenterOfMass::tmp_rho_xi");
  if(tmp_rho_xi_vi < 0) {
    CCTK_VERROR("Could not obtain variable index for CenterOfMass:tmp_rho_xi: %d",
                tmp_rho_xi_vi);
  }

  // compute M, M_x, M_y, M_z as components of [M, M_x, M_y, M_z]
  for(int c = 0 ; c < 4 ; c++) {
    // compute integrand on whole grid for all past and current timelevels
    BEGIN_REFLEVEL_LOOP(cctkGH) {
      BEGIN_LOCAL_MAP_LOOP(cctkGH, CCTK_GF) {
        BEGIN_LOCAL_COMPONENT_LOOP(cctkGH, CCTK_GF) {
          DECLARE_CCTK_ARGUMENTS; // in local mode
          const CCTK_REAL *xi[4] = {NULL,x,y,z};

          #pragma omp parallel
          CCTK_LOOP3_ALL(CenterOfMass_Compute, cctkGH, i,j,k) {
            int idx = CCTK_GFINDEX3D(cctkGH, i,j,k);

            tmp_rho_xi[idx] = rho[idx] * (xi[c] ? xi[c][idx] : 1.0);
            tmp_rho_xi_p[idx] = rho_p[idx] * (xi[c] ? xi[c][idx] : 1.0);
            tmp_rho_xi_p_p[idx] = rho_p_p[idx] * (xi[c] ? xi[c][idx] : 1.0);

          } CCTK_ENDLOOP3_ALL(CenterOfMass_Compute);
        } END_LOCAL_COMPONENT_LOOP;
      } END_LOCAL_MAP_LOOP;
    } END_REFLEVEL_LOOP;

    // integrate
    DECLARE_CCTK_ARGUMENTS; // in global mode
    CCTK_REAL *Mxi[4] = {M, Cx, Cy, Cz}; // direction c's result
    const int ierr = CCTK_Reduce(cctkGH, -1, sum_handle, 1, CCTK_VARIABLE_REAL,
                                 Mxi[c], 1, tmp_rho_xi_vi);
    if(ierr)
      CCTK_VERROR("Could not compute integral of tmp_rho_xi: %d", ierr);
  }
    
  DECLARE_CCTK_ARGUMENTS; // in global mode
  *Cx /= *M;
  *Cy /= *M;
  *Cz /= *M;
  // "sum" only uses relative cells sizes, rescale by reference cell to get
  // intgral value
  *M *= cctk_delta_space[0] * cctk_delta_space[1] * cctk_delta_space[2];
}

## src/make.code.defn

Finally let Cactus' build system know which file to compile

In [ ]:
%%writefile arrangements/EinsteinAnalysis/CenterOfMass/src/make.code.defn
# Main make.code.defn file for thorn CenterOfMass

# Source files in this directory
SRCS = compute.cc

# Subdirectories containing source files
SUBDIRS = 

# Compiling the new thorn

In [ ]:
# Make sure the new thorn is in our ThornList
# outside of this notebook one would use an editor to edit configs/sim/ThornList

contents = None
with open("configs/sim/ThornList", 'r') as f:
    contents = f.read()
if "EinsteinAnalysis/CenterOfMass" not in contents:
    with open("configs/sim/ThornList", 'a') as f:
        f.write("EinsteinAnalysis/CenterOfMass" + '\n')
print(subprocess.check_output("tail configs/sim/ThornList", 
                              shell=True, universal_newlines=True))

In [ ]:
%%bash
# compile everything

time ./simfactory/bin/sim build -j2 sim

## Cactus test cases

Cactus test cases are:

* regression tests
* small
* quick

Cactus test cases are not:

* convergence tests
* physics correctness tests

Verifying physical correctness is the author's duty when writing the code and
may be achieved by providing example parameter files and / or special "test" 
settings.

A test consists of:

* a parameter file
* expected output stored in files

Cactus compares the new and the expected output by comparing real numbers in
ASCII files, allowing a certain "fuzz", i.e. a difference of about 1.0e-12 by
default.

### How to design a test case

Test cases should be

* simple
* small
* quick
* use as few thorns as possible (simple!)
* output only a few variables (small!)
* output only norms and 1D ASCII quantities (small!)
* finish in under one minutes (quick!)

Preferably, each major feature in a thorn should be covered by a test case. If necessary, one has to introduce helper thorns that use the tested features.

Please consult the excellent
[Adding a test case](https://docs.einsteintoolkit.org/et-docs/Adding_a_test_case)
tutorial on the ET wiki by Ian Hinder for more details.

## Test parameter file

We will use use a modified version of the TOV example paramter file in
[CactusTutorial.ipynb](https://github.com/einsteintoolkit/jupyter-et/blob/master/CactusTutorial.ipynb)
to compute the center of mass of an off-center neutron star.

Thus we will modify the initial star location to be

```Python
TOVSolver::TOV_Position_x[0] = 3.5
TOVSolver::TOV_Position_y[0] = 1.5
TOVSolver::TOV_Position_z[0] = 2.0
```

and adjust the initial grid setup to be centered on the same location

```Python
CarpetRegrid2::position_x_1 = 3.5
CarpetRegrid2::position_y_1 = 1.5
CarpetRegrid2::position_z_1 = 2.0
```

Since this breaks reflection symmetry we remove the `ReflectionSymmetry` thorn
and related parameter settings. While not strictly required for the regression
test, having "recognizable" results makes it easier to track down test failures.

```Python
CoordBase::xmin = -24.0
CoordBase::ymin = -24.0
CoordBase::zmin = -24.0
CoordBase::xmax = +24.0
CoordBase::ymax = +24.0
CoordBase::zmax = +24.0
        
CoordBase::boundary_shiftout_x_lower    = 0
CoordBase::boundary_shiftout_y_lower    = 0
CoordBase::boundary_shiftout_z_lower    = 0
```

We add our new thorn

```Python
ActiveThorns = CenterOfMass
```

request output for its grid scalars

```Python
# these makes "timeseries" output via out0d_vars readable
CarpetIOASCII::compact_format = "yes"
CarpetIOASCII::one_file_per_group = yes

CarpetIOASCII::out0d_every = 1
CarpetIOASCII::out0d_vars = "CenterOfMass::center_of_mass"
```

and remove all unneded thorns and output:

* CarpetInterp, CarpetIOBasic, CarpetIOScalar, CarpetIOHDF5
* ReflectionSymmetry
* AEILocalInterp, LocalReduce
* Formaline, NaNChecker, TerminationTrigger
* 1D, 2D ASCII output, HDF5 output

Finaly since this is a test we reduce runtime to `cctk_itlast = 512` (1 full
coarse timesteps, given 10 refinement levels) and disable some output that would
produce ever changing timestamps in ASCII file comments

```Python
# Finalize
Cactus::terminate           = "iteration"
Cactus::cctk_itlast         = 512
        
IO::out_fileinfo  = "axis labels"
IO::parfile_write = "no"
```

Using a simpler ODE timestepper runs the test more quickly with fewer
right-hand-side evalutions

```Python
MoL::ODE_Method             = "Euler"
MoL::MoL_Intermediate_Steps = 1
MoL::MoL_Num_Scratch_Levels = 0
```

and some textual description that will be shown by the test system:

```Python
!DESC "Compute center of mass of an off-center TOV star"
```

In [ ]:
%%writefile arrangements/EinsteinAnalysis/CenterOfMass/test/offcenterTOV.par
!DESC "Compute center of mass of an off-center TOV star"

# Some basic stuff
ActiveThorns = "Time MoL"
ActiveThorns = "Coordbase CartGrid3d Boundary StaticConformal"
ActiveThorns = "SymBase ADMBase TmunuBase HydroBase InitBase ADMCoupling ADMMacros"
ActiveThorns = "IOUtil"
ActiveThorns = "SpaceMask CoordGauge Constants LoopControl"
ActiveThorns = "Carpet CarpetLib CarpetReduce CarpetRegrid2"
ActiveThorns = "CarpetIOASCII"

ActiveThorns = "CenterOfMass"

# Finalize
Cactus::terminate           = "iteration"
Cactus::cctk_itlast         = 512

# grid parameters
Carpet::domain_from_coordbase = "yes"
CartGrid3D::type         = "coordbase"
CartGrid3D::domain       = "full"
CartGrid3D::avoid_origin = "no"
CoordBase::xmin = -24.0
CoordBase::ymin = -24.0
CoordBase::zmin = -24.0
CoordBase::xmax = +24.0
CoordBase::ymax = +24.0
CoordBase::zmax = +24.0
# Change these parameters to change resolution. The ?max settings above
# have to be multiples of these. 'dx' is the size of one cell in x-direction.
# Making this smaller means using higher resolution, because more points will
# be used to cover the same space.
CoordBase::dx   =   2.0
CoordBase::dy   =   2.0
CoordBase::dz   =   2.0

CarpetRegrid2::regrid_every =   0
CarpetRegrid2::num_centres  =   1
CarpetRegrid2::num_levels_1 =   2
CarpetRegrid2::radius_1[1]  = 12.0
CarpetRegrid2::position_x_1 = 3.5
CarpetRegrid2::position_y_1 = 1.5
CarpetRegrid2::position_z_1 = 2.0

CoordBase::boundary_size_x_lower        = 3
CoordBase::boundary_size_y_lower        = 3
CoordBase::boundary_size_z_lower        = 3
CoordBase::boundary_size_x_upper        = 3
CoordBase::boundary_size_y_upper        = 3
CoordBase::boundary_size_z_upper        = 3
CoordBase::boundary_shiftout_x_lower    = 0
CoordBase::boundary_shiftout_y_lower    = 0
CoordBase::boundary_shiftout_z_lower    = 0
CoordBase::boundary_shiftout_x_upper    = 0
CoordBase::boundary_shiftout_y_upper    = 0
CoordBase::boundary_shiftout_z_upper    = 0

# storage and coupling
TmunuBase::stress_energy_storage = yes
TmunuBase::stress_energy_at_RHS  = yes
TmunuBase::timelevels            =  1
TmunuBase::prolongation_type     = none


HydroBase::timelevels            = 3

ADMMacros::spatial_order = 4

SpaceMask::use_mask      = "yes"

Carpet::enable_all_storage       = no
Carpet::use_buffer_zones         = "yes"

Carpet::poison_new_timelevels    = "yes"
Carpet::check_for_poison         = "no"

Carpet::init_3_timelevels        = no
Carpet::init_fill_timelevels     = "yes"

CarpetLib::poison_new_memory = "yes"
CarpetLib::poison_value      = 114

# system specific Carpet paramters
Carpet::max_refinement_levels    = 10
driver::ghost_size               = 3
Carpet::prolongation_order_space = 3
Carpet::prolongation_order_time  = 2

# Time integration
time::dtfac = 0.25

MoL::ODE_Method             = "Euler"
MoL::MoL_Intermediate_Steps = 1
MoL::MoL_Num_Scratch_Levels = 0

# Hydro paramters

ActiveThorns = "EOS_Omni GRHydro"

HydroBase::evolution_method      = "GRHydro"

GRHydro::riemann_solver         = "Marquina"
GRHydro::GRHydro_eos_type       = "Polytype"
GRHydro::GRHydro_eos_table      = "2D_Polytrope"
GRHydro::recon_method           = "ppm"
GRHydro::GRHydro_stencil        = 3
GRHydro::bound                  = "none"
GRHydro::rho_abs_min            = 1.e-10
# Parameter controlling finite difference order of the Christoffel symbols in GRHydro
GRHydro::sources_spatial_order  = 4

# Curvature evolution parameters

ActiveThorns = "GenericFD NewRad"
ActiveThorns = "ML_BSSN ML_BSSN_Helper"
ADMBase::evolution_method        = "ML_BSSN"
ADMBase::lapse_evolution_method  = "ML_BSSN"
ADMBase::shift_evolution_method  = "ML_BSSN"
ADMBase::dtlapse_evolution_method= "ML_BSSN"
ADMBase::dtshift_evolution_method= "ML_BSSN"

ML_BSSN::timelevels = 3

ML_BSSN::harmonicN           = 1      # 1+log
ML_BSSN::harmonicF           = 2.0    # 1+log
ML_BSSN::evolveA             = 1
ML_BSSN::evolveB             = 1
# NOTE: The following parameters select geodesic slicing. This choice only enables you to evolve stationary spacetimes.
#       They will not allow you to simulate a collapsing TOV star.
ML_BSSN::ShiftGammaCoeff     = 0.0
ML_BSSN::AlphaDriver         = 0.0
ML_BSSN::BetaDriver          = 0.0
ML_BSSN::advectLapse         = 0
ML_BSSN::advectShift         = 0
ML_BSSN::MinimumLapse        = 1.0e-8

ML_BSSN::my_initial_boundary_condition = "extrapolate-gammas"
ML_BSSN::my_rhs_boundary_condition     = "NewRad"

# Some dissipation to get rid of high-frequency noise
ActiveThorns = "SphericalSurface Dissipation"
Dissipation::verbose   = "no"
Dissipation::epsdis   = 0.01
Dissipation::vars = "
        ML_BSSN::ML_log_confac
        ML_BSSN::ML_metric
        ML_BSSN::ML_curv
        ML_BSSN::ML_trace_curv
        ML_BSSN::ML_Gamma
        ML_BSSN::ML_lapse
        ML_BSSN::ML_shift
"


# init parameters
InitBase::initial_data_setup_method = "init_some_levels"

# Use TOV as initial data
ActiveThorns = "TOVSolver"

HydroBase::initial_hydro         = "tov"
ADMBase::initial_data            = "tov"
ADMBase::initial_lapse           = "tov"
ADMBase::initial_shift           = "tov"
ADMBase::initial_dtlapse         = "zero"
ADMBase::initial_dtshift         = "zero"

# Parameters for initial star
tOVSolver::TOV_Position_x[0] = 3.5
TOVSolver::TOV_Position_y[0] = 1.5
TOVSolver::TOV_Position_z[0] = 2.0

TOVSolver::TOV_Rho_Central[0] = 1.28e-3
TOVSolver::TOV_Gamma          = 2
TOVSolver::TOV_K              = 100

# Set equation of state for evolution
EOS_Omni::poly_gamma                   = 2
EOS_Omni::poly_k                       = 100
EOS_Omni::gl_gamma                     = 2
EOS_Omni::gl_k                         = 100


# I/O

# Use (create if necessary) an output directory named like the
# parameter file (minus the .par)
IO::out_dir             = ${parfile}
IO::out_fileinfo        = "axis labels"
IO::parfile_write       = "no"
        
# Write one file overall per output (variable/group)
# In production runs, comment this or set to "proc" to get one file
# per MPI process
IO::out_mode            = "onefile"

# these makes "timeseries" output via out0d_vars readable
CarpetIOASCII::compact_format = "yes"
CarpetIOASCII::one_file_per_group = yes

CarpetIOASCII::out0d_every = 1
CarpetIOASCII::out0d_vars = "CenterOfMass::center_of_mass"

## Generating test data

Simfactory's `--testsuite` option, and the underlying `sim-testsuite` make
target of Cactus, can be used to run testsuites,
but cannot be conveniently used to generate the inital set of
expected output. Instead we will run Cactus manually on the test parfile.

In [ ]:
%%bash

cd arrangements/EinsteinAnalysis/CenterOfMass/test

# remove any previous stored output
rm -rf offcenterTOV

export OMP_NUM_THREADS=2
time mpirun -n 2 ../../../../exe/cactus_sim offcenterTOV.par

tail offcenterTOV/centerofmass-center_of_mass..asc

## Testing the new test

To check that the new testsuite is fully set up, we will run it once.

For this notebook we use Cactus' `sim-testsuite` target and disabling
all prompts and running only the new test and verifying that is passes
using 1 MPI rank while having been produced using 2 MPI ranks.

In [ ]:
%%bash

export OMP_NUM_THREADS=1
make sim-testsuite PROMPT=no CCTK_TESTSUITE_RUN_PROCESSORS=1 CCTK_TESTSUITE_RUN_TESTS=CenterOfMass

# Challenge problems

If you would like to play around with the code. Here are some suggestions for what to do:

* change the integrand to include $\sqrt{\det g_{ij}}$ so that `M` is the baryonic (rest) mass $M_0$
* use `TOVSolver::TOV_Velocity_x[0]` etc. to give the neutron star some initial velocity and plot how the center of mass changes as the star moves
* change to code to work if `HydroBase::rho` has only a single timelevel. What restrictions does that impose in interpreting output?
  * Hint: <span style='background-color:black; color:black'>at which times is it well defined</span>?

# Extra slides

The material below is not directly used in the tutorial, it
provides some backgroun information on how Carpet traverses the
scheddule and on the interaction between `GLOBAL` and `LOCAL`
schedule functinon.

## Mesh refinement

Mesh refinement, in particular subcylcing in time, adds complications to
computing reductions (and any other "global" quantity.).

* a (`LOCAL`) scheduled function is called once per grid patch
* different refinement levels exist at different times
* `sum` like reductions have to take relative size of grid cells into account
* during `EVOL` refinement levels are updated coarse-to-fine

### Modes

Carpet introduces the concept of "modes" to add a concept of acces levels to the
Cactus schedule:

<dl>
    <dt><code>LOCAL</code></dt><dd>called once per component, has access to
    poisition dependent data on the grid</dd>
    <dt><code>LEVEL</code></dt><dd>called once per refinement level,
    <code>SYNC</code> happens here, logically boundary conditions are speficied
    in this mode, the schedule bins are traversed in this mode. </dd>
    <dt><code>GLOBAL</code></dt><dd>called once per time step, has access to
    grid scalars and grid arrays only</dd>
</dl>

`CCTK_Reduce` can be called in either `LEVEL` or `GLOBAL` mode and will return
a reduction over all values on a *single* refinement level or *all* refinement
levels.

### Interpolation in time

Since `GLOBAL` routines are called for every since timestep, and not just every
"coarse" timestep, a reduction maybe called at times that do not correspond to a
time for which a coarse refinement level exists. In the case CarpetReduce will
interpolate grid data in time to provide coarse grid data at the required time.

### Cell volumes

Naively computing the sum of values on the grid once multiple refinement levels
exist is not very useful since such an operation does not correspond to any
continuum functional of the grid variables. Carpet this weighs each contribution
by the relative cell volume, producing an approximation of the Riemann sum. Since
Carpet does now know the physical volume that a cell corresponds to, a relative
cell volume normalized by the volume of the cells on the coarsest grid is used.

### `EVOL` vs `ANALYSIS`

01234567890123456789012345678901234567890123456789012345678901234567890123456789

Refinement levels are traversed coarse to fine, so during `EVOL` Carpet takes a
step on the coarsest level first, then a step with half the stepsize on the next
finest level, etc. meaning that data one coarse levels is *ahead* in time of the
fine levels. In `ANALYSIS` the fine refinement levels have caught up with the
coarse ones, so all refinement are at the *same* time.

In `EVOL` Carpet will call `GLOBAL` scheduled functions along with `LOCAL`
routines on the coarset level (first) so that XXX.

In `ANALYSIS` Carpet will call `GLOBAL` scheduled functiosn along with `LOCAL`
routines on the finest level (last) so that a reduction over a just computed
analysis quantity is possible.

## Sketch of schedule traversal using 2 refinement levels

```Perl
schedule take_step IN EVOL
{
    LANG: C
    OPTIONS: LOCAL
} "update to next time"

schedule compute_average IN EVOL AFTER take_step
{
    LANG: C
    OPTIONS: GLOBAL
} "compute average"

schedule compute_energy IN ANALYSIS
{
    LANG: C
    OPTIONS: GLOBAL
} "compute energy density"


schedule compute_total_energy IN ANALYSIS AFTER compute_energy
{
    LANG: C
    OPTIONS: GLOBAL
} "compute total energy"

```



<table cellspacing="0" border="0">
	<colgroup span="4" width="85"></colgroup>
	<colgroup width="123"></colgroup>
	<colgroup width="143"></colgroup>
	<colgroup width="230"></colgroup>
	<colgroup span="6" width="34"></colgroup>
	<tr>
		<td height="17" align="left" bgcolor="#B3B3B3"><b>iteration</b></td>
		<td align="left" bgcolor="#B3B3B3"><b>cctk_time</b></td>
		<td align="left" bgcolor="#B3B3B3"><b>reflevel</b></td>
		<td align="left" bgcolor="#B3B3B3"><b>mode</b></td>
		<td align="left" bgcolor="#B3B3B3"><b>bin</b></td>
		<td align="left" bgcolor="#B3B3B3"><b>function</b></td>
		<td align="left" bgcolor="#B3B3B3"><b>description</b></td>
		<td colspan=3 align="left" bgcolor="#999999"><b>reflevel 0</b></td>
		<td colspan=3 align="left" bgcolor="#CCCCCC"><b>reflevel 1</b></td>
		</tr>
	<tr>
		<td height="17" align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left">tl=0</td>
		<td align="left">tl=1</td>
		<td align="left">tl=2</td>
		<td align="left">tl=0</td>
		<td align="left">tl=1</td>
		<td align="left">tl=2</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#4C4C4C" sdval="0" sdnum="4105;"><font color="#FFFFFF">0</font></td>
		<td align="right" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#999999" sdval="0" sdnum="4105;">0</td>
		<td align="left" bgcolor="#0000FF">local</td>
		<td align="left" bgcolor="#00FF00">INITIAL</td>
		<td align="left" bgcolor="#FF00FF">intial_data</td>
		<td align="left" bgcolor="#FF00FF">set initial data</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#000000" sdval="-2" sdnum="4105;"><font color="#FFFFFF">-2</font></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
		<td align="left"><br></td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#4C4C4C" sdval="0" sdnum="4105;"><font color="#FFFFFF">0</font></td>
		<td align="right" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td align="left" bgcolor="#0000FF">local</td>
		<td align="left" bgcolor="#00FF00">INITIAL</td>
		<td align="left" bgcolor="#FF00FF">intial_data</td>
		<td align="left" bgcolor="#FF00FF">set initial data</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#000000" sdval="-2" sdnum="4105;"><font color="#FFFFFF">-2</font></td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;"><font color="#FFFFFF">-0.5</font></td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
	</tr>
	<tr>
		<td style="border-top: 2px solid #000000" height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#999999" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="left"><br></td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#00FFFF">EVOL</td>
		<td style="border-top: 2px solid #000000" align="left"><br></td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#999999">cycle timelevels, 0-&gt;1, 1-&gt;2</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#E8E8E8"><br></td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;"><font color="#FFFFFF">-0.5</font></td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#999999" sdval="0" sdnum="4105;">0</td>
		<td align="left" bgcolor="#0000FF">local</td>
		<td align="left" bgcolor="#00FFFF">EVOL</td>
		<td align="left" bgcolor="#FF950E">take_step</td>
		<td align="left" bgcolor="#FF950E">update level to time 1.00</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;"><font color="#FFFFFF">-0.5</font></td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FFFF00">global</td>
		<td align="left" bgcolor="#00FFFF">EVOL</td>
		<td align="left" bgcolor="#0000FF">compute_average</td>
		<td align="left" bgcolor="#0000FF">compute average</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;"><font color="#FFFFFF">-0.5</font></td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#B3B3B3" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#00FFFF">EVOL</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#999999">cycle timelevels, 0-&gt;1, 1-&gt;2</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="left" bgcolor="#BABABA"><br></td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;">-0.5</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#B3B3B3" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td align="left" bgcolor="#0000FF">local</td>
		<td align="left" bgcolor="#00FFFF">EVOL</td>
		<td align="left" bgcolor="#FF950E">take_step</td>
		<td align="left" bgcolor="#FF950E">update level to time 0.50</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;">-0.5</td>
	</tr>
	<tr>
		<td style="border-bottom: 2px solid #000000" height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="left"><br></td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#FFFF00">global</td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#00FFFF">EVOL</td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#0000FF">compute_average</td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#0000FF">compute average</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;">-0.5</td>
	</tr>
	<tr>
		<td style="border-top: 2px solid #000000" height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#B3B3B3" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#0000FF">local</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#FFD320">ANALYSIS</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#FFFF00">compute_energy</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#FFFF00">compute eergy density at time 0.50</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;">-0.5</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#B3B3B3" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FFFF00">global</td>
		<td align="left" bgcolor="#FFD320">ANALYSIS</td>
		<td align="left" bgcolor="#23FF23">compute_total_energy</td>
		<td align="left" bgcolor="#23FF23">compute total energy</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;">-0.5</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#B3B3B3" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td align="left" bgcolor="#FF9966">level</td>
		<td align="left" bgcolor="#FF00FF">OutputGH</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FF00FF">output level 1 data at t=0.50</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;">-0.5</td>
	</tr>
	<tr>
		<td style="border-bottom: 2px solid #000000" height="17" align="right" bgcolor="#666666" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#B3B3B3" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-bottom: 2px solid #000000" align="left"><br></td>
		<td style="border-bottom: 2px solid #000000" align="left"><br></td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#FF00FF">OutputGH</td>
		<td style="border-bottom: 2px solid #000000" align="left"><br></td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#00FFFF">output scalars at t=0.50</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#666666" sdval="-0.5" sdnum="4105;">-0.5</td>
	</tr>
	<tr>
		<td style="border-top: 2px solid #000000" height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="left"><br></td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#00FFFF">EVOL</td>
		<td style="border-top: 2px solid #000000" align="left"><br></td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#999999">cycle timelevels, 0-&gt;1, 1-&gt;2</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#E8E8E8"><br></td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td align="left" bgcolor="#0000FF">local</td>
		<td align="left" bgcolor="#00FFFF">EVOL</td>
		<td align="left" bgcolor="#FF950E">take_step</td>
		<td align="left" bgcolor="#FF950E">update level to time 1.00</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FFFF00">global</td>
		<td align="left" bgcolor="#00FFFF">EVOL</td>
		<td align="left" bgcolor="#0000FF">compute_average</td>
		<td align="left" bgcolor="#0000FF">compute average</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td style="border-bottom: 2px solid #000000" height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="left"><br></td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#00FFFF">EVOL</td>
		<td style="border-bottom: 2px solid #000000" align="left"><br></td>
		<td style="border-bottom: 2px solid #000000" align="left" bgcolor="#CCCCCC">restrict fine to coarse</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-bottom: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td style="border-top: 2px solid #000000" height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#999999" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#0000FF">local</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#FFD320">ANALYSIS</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#FFFF00">compute_energy</td>
		<td style="border-top: 2px solid #000000" align="left" bgcolor="#FFFF00">compute eergy density at time 1.00</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td style="border-top: 2px solid #000000" align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#999999" sdval="0" sdnum="4105;">0</td>
		<td align="left" bgcolor="#FF9966">level</td>
		<td align="left" bgcolor="#FF00FF">OutputGH</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FF00FF">output level 0 data at t=1.00</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td align="left" bgcolor="#0000FF">local</td>
		<td align="left" bgcolor="#FFD320">ANALYSIS</td>
		<td align="left" bgcolor="#FFFF00">compute_energy</td>
		<td align="left" bgcolor="#FFFF00">compute eergy density at time 1.00</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FFFF00">global</td>
		<td align="left" bgcolor="#FFD320">ANALYSIS</td>
		<td align="left" bgcolor="#23FF23">compute_total_energy</td>
		<td align="left" bgcolor="#23FF23">compute total energy</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#CCCCCC" sdval="1" sdnum="4105;">1</td>
		<td align="left" bgcolor="#FF9966">level</td>
		<td align="left" bgcolor="#FF00FF">OutputGH</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FF00FF">output level 1 data at t=1.00</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
	<tr>
		<td height="17" align="right" bgcolor="#B3B3B3" sdval="2" sdnum="4105;">2</td>
		<td align="right" bgcolor="#E6E6E6" sdval="1" sdnum="4105;">1</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#FFFF00">global</td>
		<td align="left" bgcolor="#FF00FF">OutputGH</td>
		<td align="left"><br></td>
		<td align="left" bgcolor="#00FFFF">output scalars at t=1.00</td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
		<td align="right" bgcolor="#4C4C4C" sdval="-1" sdnum="4105;"><font color="#FFFFFF">-1</font></td>
		<td align="right" bgcolor="#E8E8E8" sdval="1" sdnum="4105;">1</td>
		<td align="right" bgcolor="#BABABA" sdval="0.5" sdnum="4105;">0.5</td>
		<td align="right" bgcolor="#939393" sdval="0" sdnum="4105;">0</td>
	</tr>
</table>
